In [94]:
""" Classes and examples for searching for flights using SkyPicker. """
from __future__ import unicode_literals, absolute_import, generators, \
    print_function

import requests
from datetime import datetime, timedelta
import numpy as np

In [95]:
class SkyPickerApi(object):
    """ SkyPicker API. """
    def __init__(self):
        """ Initializes the API object with URL attributes. """
        self.base_url = 'https://api.skypicker.com/'
        self.path = ''
        self.param_str = ''

    @property
    def full_url(self):
        """ Returns the full URL for requesting the data. """
        return '{}{}{}'.format(self.base_url, self.path, self.param_str)

    def get_request(self):
        """ Requests the API endpoint and returns the response """
        headers = {'content-type': 'application/json'}
        resp = requests.get(self.full_url, headers=headers)
        return resp.json()
    

    def search_places(self, place_name, locale=None):
        """ Finds matching place API ids to use for searches.
        :param place_name: string of the place name to search for
        :kwarg locale: two letter lowercase locale string

        returns JSON response
        """
        self.path = 'places'
        self.param_str = '?term={}'.format(place_name)
        if locale:
            self.param_str += '&locale={}'.format(locale)
        return self.get_request()
    
    def search_airports(self, iata_code, locale=None):
        """ gets lon and lat of given airport
        returns JSON response
        """
        self.path = 'locations'
        self.param_str = '?term={}'.format(iata_code)
        if locale:
            self.param_str += '&locale={}'.format(locale)
        return self.get_request()

    def search_flights(self, origin, start_date, end_date):
        """ Searches for direct flights given a time range and origin airport code.
        :param origin: string representing the ID or IATA
        :param start_date: datetime representing first possible travel date
        :param end_date: datetime representing last possible travel date

        returns JSON response
        """
        self.path = 'flights'
        self.param_str = '?flyFrom={}&dateFrom={}&dateTo={}&directFlights={}'.format(
                origin, start_date.strftime('%d/%m/%Y'),
                end_date.strftime('%d/%m/%Y'),1)

        resp = self.get_request()
        flights = []
        for flight in resp.get('data'):
            flight_info = {
                'departure': datetime.utcfromtimestamp(flight.get('dTimeUTC')),
                'arrival': datetime.utcfromtimestamp(flight.get('aTimeUTC')),
                'price': flight.get('price'),
                'currency': resp.get('currency'),
                'legs': []
            }
            flight_info['duration'] = flight_info['arrival'] - \
                flight_info['departure']
            flight_info['duration_hours'] = (flight_info[
                'duration'].total_seconds() / 60.0) / 60.0
            for route in flight['route']:
                flight_info['legs'].append({
                    'carrier': route['airline'],
                    'departure': datetime.utcfromtimestamp(
                        route.get('dTimeUTC')),
                    'arrival': datetime.utcfromtimestamp(
                        route.get('aTimeUTC')),
                    'from': '{} ({})'.format(route['cityFrom'],
                                             route['flyFrom']),
                    'to': '{} ({})'.format(route['cityTo'], route['flyTo']),
                })
            flight_info['carrier'] = ', '.join(set([c.get('carrier') for c
                                                    in flight_info['legs']]))
            flights.append(flight_info)
        return flights

In [96]:
origin = 'KEF'    #origin airport IATA code
travel_date = '07/20/2018'   
sp_api = SkyPickerApi()
sp_results = sp_api.search_flights(origin, datetime.strptime(travel_date, '%m/%d/%Y'),
                                       datetime.strptime(travel_date, '%m/%d/%Y')+timedelta(days=7))



In [97]:
dest_iata_codes = []
for i in range(len(sp_results)):
    dest_iata_code = sp_results[i]['legs'][0]['to'][-4:-1]
    dest_iata_codes.append(dest_iata_code)
    
unique_dest_iata_codes = set(dest_iata_codes)
print(unique_dest_iata_codes)

{'HAM', 'WAW', 'GLA', 'TRS', 'CGN', 'BRE', 'VNO', 'DRS', 'MCI', 'TLV', 'CVG', 'SFO', 'GOH', 'PDX', 'CPH', 'MAD', 'ORD', 'AMS', 'RIX', 'DME', 'AGP', 'STL', 'POZ', 'YEG', 'ANC', 'MSP', 'BCN', 'EWR', 'YHZ', 'BLL', 'ZRH', 'BWI', 'DFW', 'DUB', 'ARN', 'CDG', 'BSL', 'YVR', 'KTW', 'SXF', 'GDN', 'GOT', 'BRU', 'LAX', 'TFS', 'SEA', 'FRA', 'OSL', 'YUL', 'HEL', 'GVA', 'PIT', 'PMI', 'ALC', 'MXP', 'YYZ', 'BUD', 'VIE', 'PHL', 'DEN', 'PRG', 'DTW', 'WRO', 'EDI', 'LTN', 'DUS', 'NUE', 'LYS', 'BGO', 'MUC', 'MAN', 'CLE', 'JAV', 'BOS'}


In [98]:
sp_loc_api = SkyPickerApi()
unique_dest_lons = []
unique_dest_lats = []

        


In [99]:
for iata_code in unique_dest_iata_codes:
    lat = None; lon = None;  #reset values
    airport_results = sp_api.search_airports(iata_code)
    for airport_result in airport_results['locations']:
        if airport_result['id'] != iata_code:
            continue
        print(iata_code+' '+airport_result['id'])
        lat = airport_result['location']['lat']
        lon = airport_result['location']['lon']
    unique_dest_lats.append(lat)
    unique_dest_lons.append(lon)

HAM HAM
WAW WAW
GLA GLA
TRS TRS
CGN CGN
BRE BRE
VNO VNO
DRS DRS
MCI MCI
TLV TLV
CVG CVG
SFO SFO
GOH GOH
PDX PDX
CPH CPH
MAD MAD
ORD ORD
AMS AMS
RIX RIX
DME DME
AGP AGP
STL STL
POZ POZ
YEG YEG
ANC ANC
MSP MSP
BCN BCN
EWR EWR
YHZ YHZ
BLL BLL
ZRH ZRH
BWI BWI
DFW DFW
DUB DUB
ARN ARN
CDG CDG
BSL BSL
YVR YVR
KTW KTW
SXF SXF
GDN GDN
GOT GOT
BRU BRU
LAX LAX
TFS TFS
SEA SEA
FRA FRA
OSL OSL
YUL YUL
HEL HEL
GVA GVA
PIT PIT
PMI PMI
ALC ALC
MXP MXP
YYZ YYZ
BUD BUD
VIE VIE
PHL PHL
DEN DEN
PRG PRG
DTW DTW
WRO WRO
EDI EDI
LTN LTN
DUS DUS
NUE NUE
LYS LYS
BGO BGO
MUC MUC
MAN MAN
CLE CLE
JAV JAV
BOS BOS


In [113]:
northern_airport_code = []
northern_airport_lon = []
northern_airport_lat = []
for i in range(len(unique_dest_iata_codes)):
    if unique_dest_lats[i] > 55:
        northern_airport_code.append(list(unique_dest_iata_codes)[i])
        northern_airport_lon.append(unique_dest_lons[i])
        northern_airport_lat.append(unique_dest_lats[i])

import pandas as pd
if len(northern_airport_lat) == 0:
    print("no direct routes from this airport pass near the Arctic Circle. Please try another airport")
else:
    northern_airports = pd.DataFrame(data=np.transpose([northern_airport_code,northern_airport_lon,northern_airport_lat]),
                                    columns=['code','lon','lat'])

In [114]:
northern_airports

,code,lon,lat
0,GLA,-4.433333,55.871944
1,GOH,-51.678056,64.190833
2,CPH,12.650278,55.620278
3,RIX,23.971111,56.923611
4,DME,37.906389,55.408889
5,ANC,-149.99639,61.174444
6,BLL,9.151667,55.740278
7,ARN,17.918611,59.651944
8,GOT,12.279722,57.662778
9,OSL,11.100278,60.193889


In [65]:
a = list(unique_dest_iata_codes)

In [66]:
a[2]

'GLA'